In [1]:
import utils

import importlib

importlib.reload(utils)

[2023-09-13 17:00:42,418] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


<module 'utils' from '/data/tongyx361/reward-by-prm800k/src/utils.py'>

In [2]:
import os
import evaluate
from utils import CLF_METRIC_NAMES

In [3]:
logger = utils.get_logger(__name__)

validation_set_path = os.path.join(
    utils.project_root, "datasets", "prm800k-002validation-seed42.jsonl"
)
queries_path = os.path.join(
    utils.project_root,
    "datasets",
    # "wrong-final-prm800k-002validation-seed42-openai-api-queries.jsonl",
    "prm800k-002validation-seed42-openai-api-queries.jsonl",
)

debug = {"query": False, "response": False}

<Logger __main__ (INFO)>


In [4]:
validation_set = utils.load_jsonl(validation_set_path)
print(len(validation_set))
print(validation_set[0])

1818
{'labeler': 'debabc6d-f79c-4ee5-a9db-5e284390254c', 'timestamp': '2022-07-17T17:11:05.765469', 'generation': None, 'is_quality_control_question': False, 'is_initial_screening_question': False, 'question': {'problem': 'If $x$ is a real number and $x^2 = 16$, what is the sum of all possible values of $x$?', 'ground_truth_answer': '0'}, 'label': {'steps': [{'completions': [{'text': "Let's find the square root of 16.", 'rating': 1, 'flagged': False}], 'human_completion': None, 'chosen_completion': 0}, {'completions': [{'text': "That's 4.", 'rating': 1, 'flagged': False}], 'human_completion': None, 'chosen_completion': 0}, {'completions': [{'text': 'And the negative of 4.', 'rating': -1, 'flagged': False}, {'text': 'So x could be 4.', 'rating': 0, 'flagged': False}, {'text': 'So one value of x is 4.', 'rating': 0, 'flagged': False}, {'text': 'So $x$ could be 4.', 'rating': 0, 'flagged': False}, {'text': 'And the negative of 4 is also a solution, -4.', 'rating': 1, 'flagged': False}, {'

In [5]:
queries = utils.load_jsonl(queries_path)
print(len(queries))
print(queries[0])

71
{'id': 'chatcmpl-7yF6YRIdT2ILjyv16yVGfmg3CRjRq', 'object': 'chat.completion', 'created': 1694592098, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'is appropriate in conversation, contains no inaccuracies or weirdness, and while there\'s no computation, it does advance the solution process, so it should be rated as 1. Rating: 1\nStep 2: """That\'s 4.""" Analysis: this step correctly identifies the square root of 16. It\'s appropriate in conversation, contains no inaccuracies or weirdness, and the computation is correct and can be verified, so it advances the solution process and should be rated as 1. Rating: 1\nStep 3: """And the negative of 4 is also a solution, -4.""" Analysis: this step correctly identifies that -4 is also a solution to the equation $x^2 = 16$. It\'s appropriate in conversation, contains no inaccuracies or weirdness, and the computation is correct and can be verified, so it advances the solution process and should be 

In [6]:
# assert
prompts = [q["task_prompt"] for q in queries]
# assert len(prompts) == len(
#     set(prompts)
# ), f"len(prompts) = {len(prompts)} != len(set(prompts)) = {len(set(prompts))}"

In [7]:
clf_metrics = [evaluate.load(name) for name in CLF_METRIC_NAMES] + [
    evaluate.load("roc_auc", "multiclass")
]

In [8]:
flat_preds = []
flat_refs = []
num_validation_sample = 0

num_queries = len(queries)

logger.info(f"num_queries = {num_queries}")

validation_subset = validation_set[:num_queries]
for sample_idx, (query, sample) in enumerate(zip(queries, validation_subset)):
    whole_text = query["task_prompt"] + query["choices"][0]["message"]["content"]

    steps, ratings, _ = utils.prm800k_extract_synthesized_analysis(
        whole_text,
        query_type="sar",
        # debug={"search": True}
    )

    reformatted_sample = utils.reformat_prm800k_sample(sample)
    step_ratings = reformatted_sample["step_ratings"]

    len_label = len(reformatted_sample["step_ratings"])
    if len(steps) < len_label:
        logger.warning(
            f"sample_idx {sample_idx}: {len(steps)} < {len(reformatted_sample['step_ratings'])}"
        )
        detail = f"reformatted_sample['step_ratings'] = {reformatted_sample['step_ratings']}\n\nsteps = {steps}\nratings = {ratings}\n\nwhole_text = {whole_text}"
        logger.debug(detail)
        # if len(steps) == len_label - 1:
        #     logger.info(detail)
        continue
    elif len(steps) > len_label:
        # truncate
        steps = steps[:len_label]
        ratings = ratings[:len_label]
        # step_ratings = step_ratings[:len_label]

    num_validation_sample += 1
    flat_preds.extend([int(r) for r in ratings])
    flat_refs.extend([sr["rating"] for sr in step_ratings])

    for step_idx, step_rating in enumerate(
        reformatted_sample["step_ratings"][:-1]
    ):  # ignore the last step
        # assert (
        #     steps[step_idx] == step_rating["step"]
        # ), f'steps[{step_idx}] != step_rating["step"]\n\nwhole_text = \n```\n{whole_text}\n```\n\nsteps[idx] = {steps[idx]}\n\nstep_rating["step"] = {step_rating["step"]}'
        if steps[step_idx] != step_rating["step"]:
            logger.warning(
                f'sample_idx {sample_idx}: steps[{step_idx}] != step_rating["step"]'
            )
            logger.debug(
                f'step_rating["step"] = {step_rating["step"]}\n\nsteps[idx] = {steps[step_idx]}\n\nwhole_text = \n```\n{whole_text}\n```'
            )

    if any(debug.values()):
        if debug.get("query"):
            print(whole_text)
        if debug.get("response"):
            print(steps)
            print(ratings)
            # print(analyses)
        break

assert len(flat_preds) == len(flat_refs)
logger.info(f"num_validation_sample = {num_validation_sample}")
logger.info(f"flat_preds = {flat_preds}")
logger.info(f"flat_refs  = {flat_refs}")

2023-09-13 17:00:55.208 [INFO] num_queries = 71
2023-09-13 17:00:55.212 [WARNING] sample_idx 5: 19 < 59
2023-09-13 17:00:55.214 [WARNING] sample_idx 6: steps[4] != step_rating["step"]
2023-09-13 17:00:55.215 [WARNING] sample_idx 10: 6 < 7
2023-09-13 17:00:55.217 [WARNING] sample_idx 14: 4 < 5
2023-09-13 17:00:55.219 [WARNING] sample_idx 18: steps[8] != step_rating["step"]
2023-09-13 17:00:55.221 [WARNING] sample_idx 26: steps[2] != step_rating["step"]
2023-09-13 17:00:55.222 [WARNING] sample_idx 33: 1 < 14
2023-09-13 17:00:55.225 [WARNING] sample_idx 53: steps[4] != step_rating["step"]
2023-09-13 17:00:55.226 [WARNING] sample_idx 58: steps[2] != step_rating["step"]
2023-09-13 17:00:55.228 [WARNING] sample_idx 65: 1 < 12
2023-09-13 17:00:55.229 [INFO] num_validation_sample = 66
2023-09-13 17:00:55.230 [INFO] flat_preds = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -

In [9]:
eval_metrics = utils.evaluate_clf_metrics_compute(
    clf_metrics, flat_refs, flat_preds, labels=[-1, 0, 1]
)
# print(eval_metrics)
for k, v in eval_metrics.items():
    if "-1" not in k:
        continue
    print(f"{k}: {v:.4f}")

pred_-1_num: 14.0000
ref_-1_num: 41.0000
f1_-1: 0.2545
precision_-1: 0.5000
recall_-1: 0.1707
